In [6]:
import pandas as pd
from datasets import load_dataset
from collections import OrderedDict
from IPython.display import Audio
import librosa

# List of language codes for the desired languages
language_codes = {
    "en": "English",
    "es": "Spanish",
    "ru": "Russian",
    "ja": "Japanese",
    "zh-CN": "Mandarin",
}

transcripts = []

for lang_code, lang_name in language_codes.items():
    try:
        dataset = load_dataset("mozilla-foundation/common_voice_11_0", lang_code, split="test", streaming=True)
        count = 0
        for x in dataset:
            if count >= 10:
                break
            count += 1
            transcripts.append({"language": lang_name, "sentence": x["sentence"], "audio": x["audio"]})
        print(f"Loaded transcripts for language: {lang_name}")
    except Exception as e:
        print(f"Error loading transcripts for language: {lang_name}. Error: {str(e)}")

df = pd.DataFrame(transcripts)
print(df)

Reading metadata...: 16354it [00:00, 58256.40it/s]


Loaded transcripts for language: English


Reading metadata...: 15520it [00:00, 67700.65it/s]


Loaded transcripts for language: Spanish


Reading metadata...: 9630it [00:00, 52153.68it/s]


Loaded transcripts for language: Russian


Reading metadata...: 4604it [00:00, 41701.38it/s]


Loaded transcripts for language: Japanese


Reading metadata...: 10581it [00:00, 64862.36it/s]


Loaded transcripts for language: Mandarin
      language                                           sentence  \
0      English  Joe Keaton disapproved of films, and Buster al...   
1      English                               She'll be all right.   
2      English                                                six   
3      English                         All's well that ends well.   
4      English  It is a busy market town that serves a large s...   
...        ...                                                ...   
4995  Mandarin                                    美洲拟鲽是鲽科下的一种比目鱼。   
4996  Mandarin                          岛根县立美术馆是位于日本岛根县松江市的一座美术馆。   
4997  Mandarin                                罗登贝瑞的许多概念在当时是相当先进的。   
4998  Mandarin                               环七通与东急目黑线立体交差附近为洗足站。   
4999  Mandarin                                            纪念碑总高度。   

                                                  audio  
0     {'path': 'common_voice_en_27710027.mp3', 'arra...  
1     {'path'

In [18]:
print(df[0])

KeyError: 0